In [115]:
from pytket.circuit import Circuit, Unitary1qBox, Unitary2qBox
import numpy as np
from pytket.circuit import OpType

# u1 = np.asarray([[1, 0],
#                  [0, 1]])
# u1box = Unitary1qBox(u1)

#U1 - diagnol matrix where input_angle / 2 = eigen-value = phase estimate [0,1) 
#we easily check this 
input_angle = 0.73  # angle as number of half turns
unitary_circuit = Circuit(1).U1(input_angle, 0)  # Base unitary for controlled U ops

In [116]:
from pytket.circuit import Op

base_u1 = Op.create(OpType.U1, [0.5])

In [117]:
from pytket.circuit import QControlBox
from pytket.circuit import CircBox

controlled_u1 = QControlBox(CircBox(unitary_circuit), n=2)

In [118]:
from pytket import Circuit
from pytket import OpType
from pytket.circuit.display import render_circuit_jupyter

basic_circuit = Circuit(2,1)

basic_circuit.H(0)
basic_circuit.CU1(input_angle, 0, 1)
basic_circuit.H(0)

#After second H-gate we measure and save to classical bit and read that value
basic_circuit.Measure(0,0)

render_circuit_jupyter(basic_circuit)


basic_circuit.name = "iterative_basic_arjun"


In [119]:
from pytket.extensions.nexus import Nexus
from datetime import datetime

nexus = Nexus()
my_project = nexus.new_project(name=f"Nexus iQuHack tutorial project! - {datetime.now()}")


New project created: Nexus iQuHack tutorial project! - 2024-02-03 23:37:34.582726


In [126]:
from pytket.extensions.nexus import QuantinuumConfig


configuration = QuantinuumConfig(device_name="H1-1LE", user_group="iQuHACK_2024")

In [121]:
from pytket.extensions.nexus import NexusBackend

# Then we'll create a NexusBackend using our config and the project we created
backend = NexusBackend(configuration, project=my_project)


Started using project with name: Nexus iQuHack tutorial project! - 2024-02-03 23:37:34.582726


In [122]:
# Compile the circuit in Nexus
compiled_circuit = backend.get_compiled_circuit(my_circuit, optimisation_level=2)

In [123]:
# Run the compiled circuit for 10 shots
handle = backend.process_circuit(compiled_circuit, n_shots=10)


# This will give you a ResultHandle
# As with other pytket-backends, a ResultHandle identifies a particular run of the circuit, which you can then use to keep track of the job status.
print(handle)

('fd54fb42-0f0c-46c5-ac94-5c125fb786ca', 1894211)


In [124]:
# Get an update on the status
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.ERROR: 'Circuit has errored. Check CircuitStatus.message for error message.'>, message='Circuit has encountered an error.', error_detail="<class 'pytket.extensions.quantinuum.backends.api_wrappers.QuantinuumAPIError'>: HTTP error submitting job, {'code': 63, 'message': 'Group does not belong to organization'}", completed_time=None, queued_time=None, submitted_time=datetime.datetime(2024, 2, 3, 23, 37, 57, 514810, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=datetime.datetime(2024, 2, 3, 23, 37, 58, 508993, tzinfo=datetime.timezone.utc), queue_position=None)

In [125]:
# If the job is COMPLETED we can then retrieve the result with the handle
result = backend.get_result(handle)


result.get_distribution()

ResourceFetchFailed: Failed to fetch resource with status code: None, message: Cannot retrieve result; job status is JobStatus(status=<JobStatusEnum.ERROR: 'ERROR'>, message='Circuit has encountered an error.', error_detail="<class 'pytket.extensions.quantinuum.backends.api_wrappers.QuantinuumAPIError'>: HTTP error submitting job, {'code': 63, 'message': 'Group does not belong to organization'}", completed_time=None, queued_time=None, submitted_time=datetime.datetime(2024, 2, 3, 23, 37, 57, 502405, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=datetime.datetime(2024, 2, 3, 23, 37, 58, 514184, tzinfo=datetime.timezone.utc), queue_position=None)